In [29]:
import numpy as np
import urllib3
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def open_matrix(filename='outfile_kuba.txt'):
    with open(filename, 'r') as f:
        l = [[int(float(num)) for num in line.split(' ')] for line in f]
    return l


def get_coco_labels():
    http = urllib3.PoolManager()
    r = http.request('GET', 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
    dane = r.data.decode('UTF-8')
    my_data = dane.split('\n')

    coco = {i+1: my_data[i] for i in range(0,len(my_data))}
    coco.pop(91) # remove hairbrush
    return coco

coco = get_coco_labels()

coco_readable_to_index = {value:key for key, value in coco.items()}

def get_all_detections(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][:-1]

def get_TP(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]

# not detected
def get_FN(confusion_matrix, class_name):
    return confusion_matrix[coco_readable_to_index[class_name]-1][-1]

def get_FP(confusion_matrix, class_name):
    return sum(get_all_detections(confusion_matrix, class_name)) - get_TP(confusion_matrix, class_name)



def get_precision(confusion_matrix, class_name):
    return get_TP(confusion_matrix, class_name) / get_all_detections(confusion_matrix, class_name)

def get_recall(confusion_matrix, class_name):
    return get_TP(confusion_matrix, class_name) / ((get_TP(confusion_matrix, class_name))+get_FN(confusion_matrix, class_name))

def get_F1_score(confusion_matrix, class_name):
    numerator = 2 * get_TP(confusion_matrix, class_name)
    denominator = numerator + get_FP(confusion_matrix, class_name) + get_FN(confusion_matrix, class_name)

    return numerator/denominator

def get_normalized(confusion_matrix, class_name):
    tmp = []
    for row in confusion_matrix:
        normalized_row = []
        for value in row:
            normalized_row.append(value/(sum(row)))
        tmp.append(normalized_row)

    return tmp[coco_readable_to_index[class_name]-1][coco_readable_to_index[class_name]-1]



def read_matrices(model_name):
    matrices = []
    files = os.listdir(model_name)
    for file in files:
        if file.endswith(".txt"):
            resolution = file.split("_")[0]
            if resolution.split("x")[0] != "None":
                resolution_int = int(resolution.split("x")[0]) * int(resolution.split("x")[1])
            else:
                resolution_int = 0
            matrices.append((resolution_int, resolution, open_matrix(model_name + "\\" + file)))

    matrices.sort(key = lambda x: x[0])
    return matrices


def get_normalized_for_resolutions(matrices_with_res , class_name):
    normalized_with_res = []
    for resolution in matrices_with_res:
        normalized_with_res.append((resolution[0], resolution[1], get_normalized(resolution[2],class_name)))

    return normalized_with_res

def get_f1_for_resolutions(matrices_with_res , class_name):
    f1_with_res = []
    for resolution in matrices_with_res:
        f1_with_res.append((resolution[0], resolution[1], get_F1_score(resolution[2],class_name)))

    return f1_with_res

def get_precision_for_resolutions(matrices_with_res , class_name):
    precision_with_res = []
    for resolution in matrices_with_res:
        precision_with_res.append((resolution[0], resolution[1], get_precision(resolution[2],class_name)))

    return precision_with_res

def get_recall_for_resolutions(matrices_with_res , class_name):
    recall_with_res = []
    for resolution in matrices_with_res:
        recall_with_res.append((resolution[0], resolution[1], get_recall(resolution[2],class_name)))

    return recall_with_res


def draw(resolutions_with_values, name="normalized"):
    fig =  plt.figure()
    plt.rcParams["figure.figsize"] = (20,10)
    values = [i[2] for i in resolutions_with_values]
    labels = [i[1] for i in resolutions_with_values]
    x_positions = np.arange(len(resolutions_with_values))
    plt.plot(x_positions, values, color='red',linestyle='dashed', linewidth = 3,
         marker='o', markerfacecolor='blue', markersize=12 )
    plt.xticks(x_positions, labels)
    plt.tick_params(axis="both", labelsize=18)
    plt.title(name)
    plt.xlabel("resolution", fontsize=20)
    plt.ylabel("value", fontsize=20)
    # plt.show()
    return fig




In [ ]:

matrices_with_res = read_matrices("ssd+mobilenet_v2")
normalized = get_normalized_for_resolutions(matrices_with_res, 'truck')
# x -> resolution
# y - f1 normalized etc
draw(normalized)

figures = []
pp = PdfPages('BigPDF.pdf')
for class_name in coco.values():
    n = get_normalized_for_resolutions(matrices_with_res, class_name)
    pp.savefig(draw(n, class_name))

pp.close()

In [ ]:

matrices_with_res = read_matrices("ssd+mobilenet_v2")
f1 = get_f1_for_resolutions(matrices_with_res, 'truck')
normalized = get_normalized_for_resolutions(matrices_with_res, 'blender')
precision = get_normalized_for_resolutions(matrices_with_res, 'truck')
recall = get_normalized_for_resolutions(matrices_with_res, 'truck')
# x -> resolution
# y - f1 normalized etc
draw(f1, 'f1')
draw(normalized, 'normalized')
draw(precision, 'precision')
draw(recall, 'recall')